In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)


In [3]:
dataDir='/Users/azakaria/Downloads'
#dataType='val2017'
annFile=f'/Users/azakaria/Downloads/nightowls_validation.json'

In [4]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [5]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
nms=[cat['name'] for cat in cats]

In [6]:
catIds = coco.getCatIds()

In [7]:
negative_bbox_annotations = []
imgIds = coco.getImgIds()
def remove_negative_bbox_anns(anns):
    valid_anns = []
    negative_bbox = False
    for ann in anns:
        for bbox_val in ann['bbox']:
            if bbox_val >= 0:
                #This is throwing a keyerror
                if ann['id'] in imgIds:
                    negative_bbox = True
        negative_bbox = False
        valid_anns.append(ann)
    #for v in valid_anns:
        #print(v['bbox'])
    return valid_anns

from pathlib import Path
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]
def generate_labels(numImgs):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes

    """
    # using with statement
    imgIds = coco.getImgIds()
    random.shuffle(imgIds)
    i=0
    imgIds = imgIds[:numImgs]
    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            print(f'imgId: {imgId}')
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            print(f'annIds: {annIds}')
            anns = coco.loadAnns(annIds)
            print(f'anns: {anns}')
            print(f'len(anns): {len(anns)}')
            print(filename)
            anns = remove_negative_bbox_anns(anns)
            for ann in anns:
                #filter out negative bb
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                print(x_center)
                print('--------------')
                """
                print('filename')
                print("*ann['bbox']")
                print(*ann['bbox'])
                print("img['width'], img['height']")
                print(img['width'], img['height'])
                print('x_center,y_center,width,height')
                print(x_center,y_center,width,height)
                print('--------------')
                """
                #if(int(cat_map[ann["category_id"]]) > 2):
                    #print(cat_map[ann["category_id"]])
                file.write(f'{cat_map[ann["category_id"]]}\t{x_center}\t{y_center}\t{width}\t{height}\n')
    return


In [8]:
import os
import shutil
from pathlib import Path
import os.path

def copy_images_for_dataset_dir(src_image_dir, dest_image_dir, image_extension):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        #print(src)
        #print(dest)
        #print('----------------')
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
            #print(f'{src} is a file')
        else:
            print(f'{src} is not a file')
    return


#rm -rf dataset/images/*; rm -rf dataset/labels/*
#dataset/labels
generate_labels(120)
#dataset/images
#copy_images_for_dataset_dir(Path('/Users/azakaria/Downloads/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/'), '.png')
#docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val

imgId: 7001122
annIds: [7001704]
anns: [{'occluded': False, 'difficult': False, 'bbox': [138, 217, 38, 78], 'id': 7001704, 'category_id': 1, 'image_id': 7001122, 'pose_id': 1, 'tracking_id': 7000103, 'ignore': 0, 'area': 2964, 'truncated': False}]
len(anns): 1
58c5812fbc260137e0969c8f.txt
0.1533203125
--------------
imgId: 7023776
annIds: []
anns: []
len(anns): 0
58c58329bc26013700159714.txt
imgId: 7048251
annIds: []
anns: []
len(anns): 0
58c5836dbc26013700162e2d.txt
imgId: 7037836
annIds: []
anns: []
len(anns): 0
58c5835cbc2601370016051b.txt
imgId: 7003151
annIds: []
anns: []
len(anns): 0
58c580d6bc26013448bf186b.txt
imgId: 7038295
annIds: []
anns: []
len(anns): 0
58c5835dbc26013700160748.txt
imgId: 7019273
annIds: []
anns: []
len(anns): 0
58c58321bc260137001584de.txt
imgId: 7008621
annIds: []
anns: []
len(anns): 0
58c580e2bc26013448bf33a1.txt
imgId: 7018974
annIds: []
anns: []
len(anns): 0
58c58321bc260137001583b3.txt
imgId: 7035884
annIds: [7018393]
anns: [{'occluded': None, 'diffic